In [1]:
import requests
from requests import Session
import secrets
import json
import echoMTG_api
import scryfall_api
import helper as h
from pprint import pprint as pp
import pandas as pd
import time

In [2]:
echoClient = echoMTG_api.echoMTGAPI('settings.json')
inventoryDB = echoClient.fullDataDump()
inventoryDB.set_code = inventoryDB.set_code.str.upper()
referenceJSON = json.load(open("../data/AllPrintings.json"))

{"token":"c3081aadc4524a9426a4e6542e7249779af2367e","message":"Login Successful!","status":"success"}


In [3]:
combined_inventoryDB = h.set_up(inventoryDB, referenceJSON, verbose = False)
combined_inventoryDB[combined_inventoryDB.expansion.str.contains('Promo Pack', case = "")]

,ID,echo_id,tradable,foil,price,date_acq,date_db_added,date_updated,date_deleted,multiverse_id,name,expansion,set_code,treatment,set_number
482,41220471,130275,0,0,1.00,2021-09-27 22:58:08,2021-09-27 22:58:08,2021-09-27 22:58:08,,100084133,Guardian of Faith,Promo Pack: Adventures in the Forgotten Realms,PAFR,,18
539,39628142,119773,0,0,0.00,2021-06-14 22:16:19,2021-06-14 22:16:19,2021-06-23 16:44:45,,100074275,Luminous Broodmoth,Promo Pack: Ikoria,PIKO,,21
754,39559104,127089,0,0,0.00,2021-06-12 00:16:07,2021-06-12 00:16:07,2021-06-12 00:16:25,,100080955,Shadrix Silverquill,Promo Pack: Strixhaven,PSTX,,230


In [4]:
def promo_helper(set_code, promo):
    '''
    helps to remove the P in front of the promo set names
    '''
    
    if not promo:
        return set_code
    else:
        return set_code[1:]

In [5]:
clean_inventoryDB = combined_inventoryDB.copy()
clean_inventoryDB['promo'] = clean_inventoryDB.expansion.str.contains('Promo Pack', case = False)
clean_inventoryDB['expansion'] = clean_inventoryDB.expansion.str.replace("Promo Pack: ", "")
clean_inventoryDB['set_code'] = clean_inventoryDB[['set_code', 'promo']].apply(lambda card: promo_helper(card.set_code, card.promo), axis = 1)
clean_inventoryDB[clean_inventoryDB.promo == True]

,ID,echo_id,tradable,foil,price,date_acq,date_db_added,date_updated,date_deleted,multiverse_id,name,expansion,set_code,treatment,set_number,promo
482,41220471,130275,0,0,1.00,2021-09-27 22:58:08,2021-09-27 22:58:08,2021-09-27 22:58:08,,100084133,Guardian of Faith,Adventures in the Forgotten Realms,AFR,,18,True
539,39628142,119773,0,0,0.00,2021-06-14 22:16:19,2021-06-14 22:16:19,2021-06-23 16:44:45,,100074275,Luminous Broodmoth,Ikoria,IKO,,21,True
754,39559104,127089,0,0,0.00,2021-06-12 00:16:07,2021-06-12 00:16:07,2021-06-12 00:16:25,,100080955,Shadrix Silverquill,Strixhaven,STX,,230,True


In [12]:
clean_inventoryDB.columns

Index(['ID', 'echo_id', 'tradable', 'foil', 'price', 'date_acq',
       'date_db_added', 'date_updated', 'date_deleted', 'multiverse_id',
       'name', 'expansion', 'set_code', 'treatment', 'set_number', 'promo'],
      dtype='object')

In [6]:
ids = clean_inventoryDB[['set_number', 'set_code']].drop_duplicates()

In [7]:
ids.set_code.unique()

array(['2X2', '2XM', 'A25', 'AER', 'AFC', 'AFR', 'ARB', 'AVR', 'BAB',
       'BNG', 'C15', 'C17', 'C20', 'CLB', 'DDS', 'DKA', 'DPA', 'ELD',
       'EMA', 'EMN', 'FUT', 'GRN', 'ICE', 'IKO', 'IMA', 'ISD', 'KHM',
       'KLD', 'M19', 'M20', 'M21', 'MH1', 'MH2', 'MIC', 'MID', 'MM2',
       'MM3', 'MMA', 'MMQ', 'NCC', 'NEC', 'NEO', 'NPH', 'ODY', 'OGW',
       'ONS', 'ORI', 'PD3', 'PLIST', 'PPRE', 'STX', 'RAV', 'RIX', 'RNA',
       'RTR', 'SHM', 'SLD', 'SNC', 'SOI', 'STA', 'STH', 'THS', 'TSR',
       'USG', 'V10', 'V11', 'VOW', 'WAR', 'ZEN', 'ZNC', 'ZNE', 'ZNR'],
      dtype=object)

In [8]:
clean_inventoryDB.columns

Index(['ID', 'echo_id', 'tradable', 'foil', 'price', 'date_acq',
       'date_db_added', 'date_updated', 'date_deleted', 'multiverse_id',
       'name', 'expansion', 'set_code', 'treatment', 'set_number', 'promo'],
      dtype='object')

In [10]:
class scryAPI:
    
    def __init__(self, token = ''):
        self.scryURL = "https://api.scryfall.com/"
        self.header = {}
        self.session = Session()
        self.token = token
        
    def getCollectionData(self, ids, id_type = ['collector_number','set'], query = False, verbose = False):
        # list of ids from the df to eventually join back on
        
        url = self.scryURL + "cards/collection/"
        data = []
        divisor = ids.shape[0] // 75
        missing = []
        
        if verbose:
            print(f'First four ids: {ids[:4]}',
                  f'Divisor is: {divisor}',
                  f'id_type is: {id_type}',
                  f'query set to: {query}',
                  sep = '\n')
        
        # we need to ensure that we are being polite with our API requests
        # scryfall requests you put 50-100ms delay, or do not average more than
        # 10 requests per second.
        # This should do ~1k cards in a little under 30 seconds.
        
        # There should be a saved list of the keys that you can query that are
        # checked before you actually get to pull the information. Alternately
        # check the docs for if there is a way to just pull specific information
        # so we are more curtious with our queries.
        
        
        # I wrote this wrong originally, query should be handled by a different function
        # or at the end of processing since we cannot return data in a meaningful way
        # 
        ids.columns = id_type
        
        if 'collector_number' in id_type:
            ids.collector_number = ids.collector_number.astype(str)
        
        ids = ids.loc[::,['set', 'collector_number']]
        
        while divisor > 0:
            current_ids = ids.iloc[:75,::]
            ids = ids.iloc[75:,::]
            
            if verbose:
                print(f'Length of current_ids: {current_ids.shape[0]}',
                      f'Length of ids: {ids.shape[0]}',
                      f'Missing IDS so far: {missing}',
                      #f'Column order: {current_ids.columns}',
                      sep = '\n')
            
            parameters = {
                'identifiers' : list(current_ids.to_dict(orient = 'index').values())
            }
            if query:
                r = self.session.post(url, json=parameters)
                data.extend(r.json()['data'].get(query, 'Category Not Found'))
                missing.extend(r.json().get('not_found', ''))
            else:
                r = self.session.post(url, json=parameters)
                data.extend(r.json()['data'])
                missing.extend(r.json().get('not_found', ''))
                
            
            divisor -= 1
            time.sleep(2)
        
        parameters = {
            'identifiers' : list(ids.to_dict(orient = 'index').values())
        }
        
        if query:
            r = self.session.post(url, json = parameters)
            data.extend(r.json()['data'])
            missing.extend(r.json().get('not_found', ''))

        else:
            r = self.session.post(url, json = parameters)
            if verbose:
                print(f'Parameters : {parameters}')
                # print('API post output', sep = ": ")
                # pp(r.json()['data'])
            data.extend(r.json()['data'])
            missing.extend(r.json().get('not_found', ''))

        if verbose:
            pp(missing)
        return data

In [9]:
# two things to consider when pushing this information through:
# set_number, set_code, foil, promo
# 
sfAPI = scryfall_api.scryAPI("na")

In [256]:
# sAPI = scryAPI()

In [10]:
output = sfAPI.getCollectionData(ids, verbose = True)

First four ids:    set_number set_code
0         360      2X2
1         258      2X2
2         303      2X2
3         363      2X2
Divisor is: 7
id_type is: ['collector_number', 'set']
query set to: False
Length of current_ids: 75
Length of ids: 462
Missing IDS so far: []
Length of current_ids: 75
Length of ids: 387
Missing IDS so far: []
Length of current_ids: 75
Length of ids: 312
Missing IDS so far: [{'set': 'BAB', 'collector_number': '112'}, {'set': 'BAB', 'collector_number': '491'}, {'set': 'BAB', 'collector_number': '234'}]
Length of current_ids: 75
Length of ids: 237
Missing IDS so far: [{'set': 'BAB', 'collector_number': '112'}, {'set': 'BAB', 'collector_number': '491'}, {'set': 'BAB', 'collector_number': '234'}]
Length of current_ids: 75
Length of ids: 162
Missing IDS so far: [{'set': 'BAB', 'collector_number': '112'}, {'set': 'BAB', 'collector_number': '491'}, {'set': 'BAB', 'collector_number': '234'}]
Length of current_ids: 75
Length of ids: 87
Missing IDS so far: [{'set': '

In [258]:
outputDB = pd.DataFrame(output)

In [259]:
outputDB.columns

Index(['object', 'id', 'oracle_id', 'multiverse_ids', 'tcgplayer_id',
       'cardmarket_id', 'name', 'lang', 'released_at', 'uri', 'scryfall_uri',
       'layout', 'highres_image', 'image_status', 'image_uris', 'mana_cost',
       'cmc', 'type_line', 'oracle_text', 'power', 'toughness', 'colors',
       'color_identity', 'keywords', 'legalities', 'games', 'reserved', 'foil',
       'nonfoil', 'finishes', 'oversized', 'promo', 'reprint', 'variation',
       'set_id', 'set', 'set_name', 'set_type', 'set_uri', 'set_search_uri',
       'scryfall_set_uri', 'rulings_uri', 'prints_search_uri',
       'collector_number', 'digital', 'rarity', 'flavor_text', 'card_back_id',
       'artist', 'artist_ids', 'illustration_id', 'border_color', 'frame',
       'frame_effects', 'full_art', 'textless', 'booster', 'story_spotlight',
       'promo_types', 'edhrec_rank', 'penny_rank', 'preview', 'prices',
       'related_uris', 'purchase_uris', 'mtgo_id', 'security_stamp',
       'tcgplayer_etched_id', 'p

In [260]:
# I guess we're just going to end up attaching everything...?
# which means we've got to define some kind of schema for attaching the prices back, should be ez
# join w/ 2 keys
price_list = outputDB.loc[::,['name', 'set', 'collector_number', 'prices']]
price_list.set = price_list.set.str.upper()
price_list.collector_number = price_list.collector_number.astype(int)
price_list

,name,set,collector_number,prices
0,Mentor of the Meek,2X2,340,"{'usd': '0.81', 'usd_foil': '1.32', 'usd_etche..."
1,Spell Pierce,2X2,350,"{'usd': '0.68', 'usd_foil': '1.43', 'usd_etche..."
2,Anger of the Gods,2X2,102,"{'usd': '0.54', 'usd_foil': '0.77', 'usd_etche..."
3,Gravecrawler,2X2,78,"{'usd': '4.31', 'usd_foil': '5.10', 'usd_etche..."
4,Thousand-Year Storm,2X2,390,"{'usd': '4.57', 'usd_foil': '5.27', 'usd_etche..."
...,...,...,...,...
460,Ancient Greenwarden,ZNR,178,"{'usd': '8.41', 'usd_foil': '8.57', 'usd_etche..."
461,Lotus Cobra,ZNR,193,"{'usd': '1.83', 'usd_foil': '2.26', 'usd_etche..."
462,"Yasharn, Implacable Earth",ZNR,240,"{'usd': '0.54', 'usd_foil': '0.62', 'usd_etche..."
463,"Sea Gate Restoration // Sea Gate, Reborn",ZNR,76,"{'usd': '15.64', 'usd_foil': '16.00', 'usd_etc..."


In [261]:
price_list.loc[0,'prices']

{'usd': '0.81',
 'usd_foil': '1.32',
 'usd_etched': None,
 'eur': '0.32',
 'eur_foil': '1.32',
 'tix': None}

In [262]:
import warnings
import numpy as np

def promo_helper(set_code, promo):
    '''
    helps to remove the P in front of the promo set names
    '''
    
    if not promo:
        return set_code
    else:
        return set_code[1:]
    
def price_lookup(card_info, priceDB, verbose = False):
    # foil, or etched foil is the only thing I care about
    
    
    foil_flag = card_info.foil == 1
    etched_flag = 'etched' in card_info.treatment.lower()
 
    filtered_prices = priceDB[priceDB['name'] == card_info['name']]
    if verbose:
        print("*"*50)
        print(f'Card name: {card_info["name"]}')
        print(f'Size of filtered_prices: {filtered_prices.shape[0]}')
        print(f'Price Collector Number: {filtered_prices.collector_number}')
        print(f'Inventory Collector Number: {card_info.set_number}')
        print(f'foil_flag: {foil_flag}')
        print(f'etched_flag: {etched_flag}')
    
    if card_info.set_code in ['BAB', 'PPRE']:
        
        warnings.warn("Not implemented for BAB or PPRE yet.")
        
        card_info['price'] = np.nan
        
        return card_info
    
    if card_info.promo:
        print("Remember to implement grabbing the promo-specific ones from scryfall.")
    
    if filtered_prices.shape[0] == 0:
        filtered_prices = priceDB[priceDB.name.str.contains(card_info['name'])]
        
        if filtered_prices.shape[0] == 0:
            filtered_prices = priceDB[priceDB.name.str.lower() == card_info['name'].lower()]
    
    specific_price = filtered_prices[filtered_prices['collector_number'] == card_info.set_number]
    
    if etched_flag:
        key = 'usd_etched'
    elif foil_flag:
        key = 'usd_foil'
    else:
        key = 'usd'
    
    if verbose:
        print('specific_price:')
        #pp(specific_price)
        print(f"specific_price['prices']: {specific_price.loc[::,'prices']}")
        print(f"Returned price: {specific_price.iloc[0].loc['prices'][key]}",
              f"Retrieved via key: {key}",
              sep="\n")
        print("*"*50)
    
    card_info['price'] = specific_price.iloc[0].loc['prices'][key]
    
    return card_info

In [263]:
clean_inventoryDB

,ID,echo_id,tradable,foil,price,date_acq,date_db_added,date_updated,date_deleted,multiverse_id,name,expansion,set_code,treatment,set_number,promo
6,45125035,138845,0,1,0.75,2022-07-13 16:04:18,2022-07-13 16:04:18,2022-07-13 16:04:18,,100092284,Mentor of the Meek,Double Masters 2022,2X2,Borderless,340,False
7,45125029,138449,0,1,0.90,2022-07-13 16:03:54,2022-07-13 16:03:54,2022-07-13 16:03:54,,100091888,Spell Pierce,Double Masters 2022,2X2,Borderless,350,False
8,45125022,138175,0,1,0.67,2022-07-13 16:03:18,2022-07-13 16:03:18,2022-07-13 16:03:18,,100091615,Anger of the Gods,Double Masters 2022,2X2,,102,False
9,45125018,138151,0,1,2.50,2022-07-13 16:02:54,2022-07-13 16:02:54,2022-07-13 16:02:54,,100091591,Gravecrawler,Double Masters 2022,2X2,,78,False
11,45124992,138434,0,1,3.00,2022-07-13 15:58:16,2022-07-13 15:58:16,2022-07-13 15:58:16,,100091873,Thousand-Year Storm,Double Masters 2022,2X2,Borderless,390,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612,39627409,121664,0,0,11.30,2021-06-14 21:47:14,2021-06-14 21:47:14,2021-06-14 21:47:14,,100076166,Ancient Greenwarden,Zendikar Rising,ZNR,,178,False
615,39627404,121678,0,0,2.04,2021-06-14 21:47:12,2021-06-14 21:47:12,2021-06-14 21:47:12,,100076180,Lotus Cobra,Zendikar Rising,ZNR,,193,False
618,39627403,121725,0,0,0.28,2021-06-14 21:47:11,2021-06-14 21:47:11,2021-06-14 21:47:11,,100076227,"Yasharn, Implacable Earth",Zendikar Rising,ZNR,,240,False
619,39627399,121564,0,0,12.11,2021-06-14 21:47:10,2021-06-14 21:47:10,2021-06-14 21:47:10,,100076066,Sea Gate Restoration,Zendikar Rising,ZNR,,76,False


In [272]:
clean_inventoryDB.foil = clean_inventoryDB.foil.astype(int)
completed_inventory = clean_inventoryDB.apply(lambda card: price_lookup(card, price_list, verbose = False), axis = 1)

/var/folders/cn/9ny0n35x1p18h680wlz6lmzm0000gn/T/ipykernel_23306/2390124132.py:33: UserWarning: Not implemented for BAB or PPRE yet.
  warnings.warn("Not implemented for BAB or PPRE yet.")


Remember to implement grabbing the promo-specific ones from scryfall.
Remember to implement grabbing the promo-specific ones from scryfall.
Remember to implement grabbing the promo-specific ones from scryfall.


In [ ]:
# We did it! refactor + moving this to other folders and fixing the files will be good

In [273]:
completed_inventory[completed_inventory.name.str.contains('Ulamog')]

,ID,echo_id,tradable,foil,price,date_acq,date_db_added,date_updated,date_deleted,multiverse_id,name,expansion,set_code,treatment,set_number,promo
51,45049210,138429,0,1,64.91,2022-07-09 12:54:34,2022-07-09 12:54:34,2022-07-09 12:54:34,,100091868,"Ulamog, the Infinite Gyre",Double Masters 2022,2X2,Borderless,337,False
52,45049209,138429,0,0,49.19,2022-07-09 12:54:33,2022-07-09 12:54:33,2022-07-09 12:54:33,,100091868,"Ulamog, the Infinite Gyre",Double Masters 2022,2X2,Borderless,337,False


In [266]:
price_list[price_list.name == 'Dauntless Escort']['prices'].iloc[0]

{'usd': None,
 'usd_foil': None,
 'usd_etched': '1.85',
 'eur': None,
 'eur_foil': None,
 'tix': None}